In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathstuff import get_path
path=get_path()


In [ ]:
import torchaudio
import torch
from pathlib import Path
from models.dataset import N2NDataset
from models.noise2noiseunet import Noise2NoiseUNet
from models.train_test import train,loss_function
from torch.utils.data import DataLoader
import noise_addition_utils
import numpy as np
from models.metrics import AudioMetrics
from models.metrics import AudioMetrics2

In [ ]:
np.random.seed(999)
torch.manual_seed(999)

# If running on Cuda set these 2 for determinism
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
torchaudio.set_audio_backend("soundfile")
input_train_dir="../Datasets/WhiteNoise_Train_Input"
output_train_dir="../Datasets/WhiteNoise_Train_Output"


In [6]:
sample_rate = 48000
n_fft = (sample_rate * 64) // 1000 
hop_length = (sample_rate * 16) // 1000 


In [7]:

train_input =list(Path(input_train_dir).iterdir())
train_target = list(Path(output_train_dir).iterdir())

val_noisy_input = list(Path("../Datasets/WhiteNoise_Test_Input").iterdir())
val_clean_output = list(Path('../Datasets/clean_testset_wav').iterdir()) 

In [8]:
train_on_gpu=torch.cuda.is_available()      
DEVICE = torch.device('cuda' if train_on_gpu else 'cpu')

In [9]:
train_data=N2NDataset(train_input, train_target, n_fft, hop_length)
train_dataloader = DataLoader(train_data, batch_size=2, shuffle=True)

val_data=N2NDataset(val_noisy_input,val_clean_output, n_fft, hop_length)
val_dataloader = DataLoader(val_data, batch_size=2, shuffle=True)

In [11]:
n2n = Noise2NoiseUNet(n_fft, hop_length).to(DEVICE)
optimizer = torch.optim.Adam(n2n.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [ ]:
loss_per_epoch,val_loss = train(n2n, train_dataloader,val_dataloader,loss_function,optimizer,n_fft, hop_length,scheduler)

In [ ]:
test_input = list(Path("Samples/Sample_Test_Input").iterdir())
test_output = list(Path("Samples/Sample_Test_Target").iterdir())

test_dataset = N2NDataset(test_input, test_output, n_fft, hop_length)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
n2n.eval()
test_dataloader= iter(test_loader)
nosiy_inp, clean_out = next(test_dataloader)

for _ in range(4):
    nosiy_inp, clean_out = next(test_dataloader)
pred_out = n2n(nosiy_inp, is_istft=True)

In [ ]:
pred_out_np = pred_out[0].view(-1).detach().cpu().numpy()
clean_out=torch.view_as_complex(clean_out)
nosiy_inp=torch.view_as_complex(nosiy_inp)
clean_out_np = torch.istft(torch.squeeze(clean_out[0], 1), n_fft=n_fft, hop_length=hop_length, normalized=True).view(-1).detach().cpu().numpy()
nosiy_inp_np = torch.istft(torch.squeeze(nosiy_inp[0], 1), n_fft=n_fft, hop_length=hop_length, normalized=True).view(-1).detach().cpu().numpy()

In [ ]:
metrics = AudioMetrics(clean_out_np, pred_out_np, sample_rate)
print(metrics.display())

In [ ]:

def save_audio(np_array=np.array([0.5]*1000),file_path='./sample_audio.wav', sample_rate=48000, bit_precision=16):
    np_array = np.reshape(np_array, (1,-1))
    torch_tensor = torch.from_numpy(np_array)
    torchaudio.save(file_path, torch_tensor, sample_rate, bits_per_sample=bit_precision)

In [ ]:
save_audio(np_array=pred_out_np,file_path=str(Path("../Samples/denoised.wav")), sample_rate=sample_rate, bit_precision=16)
save_audio(np_array=clean_out_np,file_path=str(Path("../Samples/clean.wav")), sample_rate=sample_rate, bit_precision=16)
save_audio(np_array=nosiy_inp_np,file_path=str(Path("../Samples/noisy.wav")), sample_rate=sample_rate, bit_precision=16)